
# Data Cleaning Notebook (Simplified)
This notebook:
- Uses only **pandas** and **sqlite3**
- Loads tables from `cademycode.db`
- Cleans data (types, nulls, duplicates)
- Validates join keys
- Saves a clean SQLite database and a CSV file


In [15]:

import sqlite3
import pandas as pd
import os

# Display settings for pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)


## 1. Connect to Database and Load Tables

In [ ]:

# Connect to the original database
con = sqlite3.connect(r'C:\Users\zadeboye\Documents\Database\Course_data_analysis\dev\cademycode.db')

# Find table names
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", con)
print("Tables in DB:", tables['name'].tolist())


# Load each table into a DataFrame
cademycode_students = pd.read_sql_query(f"SELECT * FROM {tables['name'].iloc[0]};", con)
student_jobs = pd.read_sql_query(f"SELECT * FROM {tables['name'].iloc[2]};", con)
courses = pd.read_sql_query(f"SELECT * FROM {tables['name'].iloc[1]};", con)

con.close()


Tables in DB: ['cademycode_students', 'cademycode_courses', 'cademycode_student_jobs']


## 2. Inspect the Tables

In [40]:

# Look at first few rows and data summaries
display(cademycode_students.head())
display(cademycode_students.info())

display(student_jobs.head())
display(student_jobs.info())

display(courses.head())
display(courses.info())


,student_id,name,dob,sex,job_id,num_course_taken,current_career_path_id,time_spent_hrs,mailing_address,email
0,1,Annabelle Avery,1943-07-03,F,7,6,1,4.99,"303 N Timber Key, Irondale, Wisconsin, 84736",annabelle_avery9376@woohoo.com
1,2,Micah Rubio,1991-02-07,M,7,5,8,4.40,"767 Crescent Fair, Shoals, Indiana, 37439",rubio6772@hmail.com
2,3,Hosea Dale,1989-12-07,M,7,8,8,6.74,"P.O. Box 41269, St. Bonaventure, Virginia, 83637",hosea_dale8084@coldmail.com
3,4,Mariann Kirk,1988-07-31,F,6,7,9,12.31,"517 SE Wintergreen Isle, Lane, Arkansas, 82242",kirk4005@hmail.com
4,5,Lucio Alexander,1963-08-31,M,7,14,3,5.64,"18 Cinder Cliff, Doyles borough, Rhode Island,...",alexander9810@hmail.com


<class 'pandas.core.frame.DataFrame'>
Index: 4293 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   student_id              4293 non-null   int64         
 1   name                    4293 non-null   object        
 2   dob                     4293 non-null   datetime64[ns]
 3   sex                     4293 non-null   object        
 4   job_id                  4293 non-null   Int64         
 5   num_course_taken        4293 non-null   Int64         
 6   current_career_path_id  4293 non-null   Int64         
 7   time_spent_hrs          4293 non-null   float64       
 8   mailing_address         4293 non-null   object        
 9   email                   4293 non-null   object        
dtypes: Int64(3), datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 381.5+ KB


None

,job_id,job_category,avg_salary
0,1,analytics,86000
1,2,engineer,101000
2,3,software developer,110000
3,4,creative,66000
4,5,financial services,135000


<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   job_id        10 non-null     int64 
 1   job_category  10 non-null     object
 2   avg_salary    10 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 320.0+ bytes


None

,career_path_id,career_path_name,hours_to_complete
0,1,data scientist,20
1,2,data engineer,20
2,3,data analyst,12
3,4,software engineering,25
4,5,backend engineer,18


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   career_path_id     10 non-null     int64 
 1   career_path_name   10 non-null     object
 2   hours_to_complete  10 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 372.0+ bytes


None

## 3. Clean Data

In [ ]:

# Example cleaning: fix types, handle nulls, remove duplicates

# Convert date columns to datetime
cademycode_students['dob'] = pd.to_datetime(cademycode_students['dob'], errors='coerce')

# Convert numeric columns
cademycode_students['time_spent_hrs'] = pd.to_numeric(cademycode_students['time_spent_hrs'], errors='coerce')
cols = ['job_id', 'num_course_taken', 'current_career_path_id']
cademycode_students[cols] = cademycode_students[cols].apply(pd.to_numeric, errors='coerce').astype('Int64')

# Drop duplicates in student_jobs
student_jobs = student_jobs.drop_duplicates()

#Drop rows with missing job_id, current_career_path_id, time_spent_hrs, num_course_taken - less than 20% of the data
cademycode_students = cademycode_students.dropna(subset=['job_id'])
cademycode_students = cademycode_students.dropna(subset=['current_career_path_id', 'time_spent_hrs', 'num_course_taken'])

# Extract email and phone from JSON-like contact_info column if exists
if 'contact_info' in cademycode_students.columns:
    contact_df = cademycode_students['contact_info'].apply(lambda x: pd.Series(eval(x)) if pd.notna(x) else pd.Series({'email': None, 'phone': None}))
    cademycode_students = pd.concat([cademycode_students.drop(columns=['contact_info']), contact_df], axis=1)

#Rename columns for clarity
cademycode_students.rename(columns={'uuid':'student_id'}, inplace=True)

# Drop rows with obviously invalid job_id or course_id
student_jobs = student_jobs[student_jobs['job_id'].notna()]
cademycode_students = cademycode_students[cademycode_students['student_id'].notna()]


## 4. Validate Relationships

In [43]:

# Ensure job_ids match between tables
valid_job_ids = set(student_jobs['job_id'])
invalid_jobs = cademycode_students.loc[~cademycode_students['job_id'].isin(valid_job_ids)]
print("Rows in student_jobs with invalid job_id:", len(invalid_jobs))


#Ensure Career Path IDs match between tables
valid_career_path_ids = set(cademycode_students['current_career_path_id'])
invalid_career_paths = courses.loc[~courses['career_path_id'].isin(valid_career_path_ids)]
print("Rows in student_jobs with invalid current_career_path_id:", len(invalid_career_paths))

# Basic uniqueness check
print("Unique student_id count:", cademycode_students['student_id'].nunique())
print("Total student_id rows:", len(cademycode_students))


Rows in student_jobs with invalid job_id: 0
Rows in student_jobs with invalid current_career_path_id: 0
Unique student_id count: 4293
Total student_id rows: 4293


## 5. Save Clean Data

In [ ]:

# Save cleaned tables to a new SQLite database
con_clean = sqlite3.connect(r'C:\Users\zadeboye\Documents\Database\Course_data_analysis\prod\cleaned_cademycode_dev.db')
cademycode_students.to_sql('cademycode_students_clean', con_clean, if_exists='replace', index=False)
student_jobs.to_sql('student_jobs_clean', con_clean, if_exists='replace', index=False)
courses.to_sql('courses_clean', con_clean, if_exists='replace', index=False)
con_clean.close()

# Optionally create a flat table CSV for analysts (example join on student_id)
flat_table = cademycode_students.merge(student_jobs, on='job_id', how='left')
flat_table = flat_table.merge(courses, left_on='current_career_path_id', right_on='career_path_id', how='left')
flat_table.to_csv(os.path.join(r'C:\Users\zadeboye\Documents\Database\Course_data_analysis\prod', 'clean_data.csv'), index=False)

print("Clean database and CSV have been saved.")


Clean database and CSV have been saved.
